In [1]:
import requests

url = "https://api.waqi.info/feed/here/"
token = "f72812a3dce2b2328992b2b06e42f4bc8e9f30da"

request_url = f"{url}?token={token}"
# Fetch the data from the API
response = requests.get(request_url)

# Check if the response is successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
    if data.get("status") == "ok":
        print("Air Quality Data:")
        print(data["data"])
    else:
        print("Error fetching data:", data.get("data"))
else:
    print(f"Failed to connect. Status code: {response.status_code}")


Air Quality Data:
{'aqi': 294, 'idx': 2554, 'attributions': [{'url': 'http://cpcb.nic.in/', 'name': 'CPCB - India Central Pollution Control Board', 'logo': 'India-CPCB.png'}, {'url': 'http://dpccairdata.com/', 'name': 'Delhi Pollution Control Commitee (Government of NCT of Delhi)', 'logo': 'India-DPCCC.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [28.6341, 77.2005], 'name': 'Mandir Marg, Delhi, Delhi, India', 'url': 'https://aqicn.org/city/delhi/mandir-marg', 'location': ''}, 'dominentpol': 'pm25', 'iaqi': {'co': {'v': 8.2}, 'dew': {'v': 14}, 'h': {'v': 86.65}, 'no2': {'v': 36}, 'o3': {'v': 15.6}, 'p': {'v': 985.95}, 'pm10': {'v': 189}, 'pm25': {'v': 294}, 'so2': {'v': 3.7}, 't': {'v': 29.025000000000002}, 'w': {'v': 0.22499999999999998}, 'wd': {'v': 75.3}}, 'time': {'s': '2024-11-04 11:00:00', 'tz': '+05:30', 'v': 1730718000, 'iso': '2024-11-04T11:00:00+05:30'}, 'forecast': {'daily': {'o3': [{'avg': 7, 'day': '2024-11-02', 'max': 37

In [4]:
import requests
import pandas as pd

# API endpoint and token
url = "https://api.waqi.info/feed/here/"
token = "f72812a3dce2b2328992b2b06e42f4bc8e9f30da"

# Create the request URL
request_url = f"{url}?token={token}"

# Fetch the data from the API
response = requests.get(request_url)

# Check if the response is successful
if response.status_code == 200:
    data = response.json()  # Parse JSON response
    if data.get("status") == "ok":
        air_quality_data = data["data"]

        # Extract input features (weather and pollutants)
        iaqi_data = air_quality_data.get("iaqi", {})
        features = {
            "temperature": iaqi_data.get("t", {}).get("v", None),
            "humidity": iaqi_data.get("h", {}).get("v", None),
            "pressure": iaqi_data.get("p", {}).get("v", None),
            "wind_speed": iaqi_data.get("w", {}).get("v", None),
            "wind_direction": iaqi_data.get("wd", {}).get("v", None),
            "pm25": iaqi_data.get("pm25", {}).get("v", None),
            "pm10": iaqi_data.get("pm10", {}).get("v", None),
            "no2": iaqi_data.get("no2", {}).get("v", None),
            "so2": iaqi_data.get("so2", {}).get("v", None),
            "o3": iaqi_data.get("o3", {}).get("v", None),
            "co": iaqi_data.get("co", {}).get("v", None)
        }

        # Extract target (AQI)
        target = air_quality_data.get("aqi", None)


        # Convert to a DataFrame for easy manipulation (optional)
        data_frame = pd.DataFrame([features])
        data_frame['target_aqi'] = target
        data_frame.to_csv("check.csv")

    else:
        print("Error fetching data:", data.get("data"))
else:
    print(f"Failed to connect. Status code: {response.status_code}")


In [6]:
# mkdir air_quality_feature_store
# cd air_quality_feature_store
# feast init


In [9]:
from datetime import datetime
from feast import FeatureStore, Entity, Feature, ValueType
import pandas as pd

# Set up the connection to the feature store
store = FeatureStore(repo_path="/home/jellyfish/Music/Air Quality Index Predictor/air_quality_feature_store/set_cod/feature_repo/")

# Define an entity for the feature store (e.g., a location ID)
location = Entity(
    name="location",
    join_keys=["location_id"],
    value_type=ValueType.STRING,
    description="Identifier for different locations"
)

from feast import FeatureView, Field
from feast.types import Float32

air_quality_feature_view = FeatureView(
    name="air_quality_features",
    entities=["location"],
    ttl=None,
    schema=[
        Field(name="temperature", dtype=Float32),
        Field(name="humidity", dtype=Float32),
        Field(name="pressure", dtype=Float32),
        Field(name="wind_speed", dtype=Float32),
        Field(name="wind_direction", dtype=Float32),
        Field(name="pm25", dtype=Float32),
        Field(name="pm10", dtype=Float32),
        Field(name="no2", dtype=Float32),
        Field(name="so2", dtype=Float32),
        Field(name="o3", dtype=Float32),
        Field(name="co", dtype=Float32)
    ],
    online=True,
    batch_source=None,  # Specify your batch source here
    tags={}
)

# Register the feature view
store.apply([location, air_quality_feature_view])

# Create a DataFrame for ingestion
data = pd.DataFrame([{
    "location_id": "delhi_mandir_marg",
    "event_timestamp": datetime.utcnow(),
    "temperature": 29.025,
    "humidity": 86.65,
    "pressure": 985.95,
    "wind_speed": 0.225,
    "wind_direction": 75.3,
    "pm25": 294,
    "pm10": 189,
    "no2": 36,
    "so2": 3.7,
    "o3": 15.6,
    "co": 8.2
}])

# Ingest data into the feature store
store.write_to_online_store("air_quality_features", data)

print("Data successfully ingested into the feature store.")


TypeError: FeatureView.__init__() got an unexpected keyword argument 'batch_source'

In [10]:
import pandas as pd
import requests
from datetime import datetime
import time
import os

# Define the API endpoint and token
API_URL = "https://api.waqi.info/feed/here/?token=f72812a3dce2b2328992b2b06e42f4bc8e9f30da"
CSV_FILE_PATH = "air_quality_feature_store.csv"

# Function to fetch data from the API
def fetch_air_quality_data():
    try:
        response = requests.get(API_URL)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
        
        data = response.json()
        if 'data' in data and data['data'] is not None:
            iaqi = data['data'].get('iaqi', {})
            return {
                "location_id": "delhi_mandir_marg",
                "event_timestamp": datetime.utcnow(),
                "temperature": iaqi.get('t', {}).get('v', None),
                "humidity": iaqi.get('h', {}).get('v', None),
                "pressure": iaqi.get('p', {}).get('v', None),
                "wind_speed": iaqi.get('w', {}).get('v', None),
                "wind_direction": iaqi.get('wd', {}).get('v', None),
                "pm25": iaqi.get('pm25', {}).get('v', None),
                "pm10": iaqi.get('pm10', {}).get('v', None),
                "no2": iaqi.get('no2', {}).get('v', None),
                "so2": iaqi.get('so2', {}).get('v', None),
                "o3": iaqi.get('o3', {}).get('v', None),
                "co": iaqi.get('co', {}).get('v', None)
            }
        else:
            print("No data found in API response.")
            return None
    except requests.exceptions.RequestException as e:
        print("Error fetching data:", e)
        return None

# Function to append data to CSV
def append_data_to_csv(data_dict, file_path):
    # Check if the file exists to write the header only if needed
    file_exists = os.path.isfile(file_path)
    df = pd.DataFrame([data_dict])
    
    try:
        if not file_exists:
            df.to_csv(file_path, index=False, mode='w')  # Create new file with header
        else:
            df.to_csv(file_path, index=False, mode='a', header=False)  # Append without header
    except Exception as e:
        print("Error writing to CSV:", e)

# Main loop to fetch data every hour and update CSV
while True:
    new_data = fetch_air_quality_data()
    if new_data:
        append_data_to_csv(new_data, CSV_FILE_PATH)
        print("New data appended at:", new_data["event_timestamp"])
    
    # Wait for 1 hour (3600 seconds)
    time.sleep(3600)


New data appended at: 2024-11-04 06:49:33.583468


KeyboardInterrupt: 

In [12]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import os

# Define the API endpoint and token
API_URL = "https://api.waqi.info/feed/here/?token=f72812a3dce2b2328992b2b06e42f4bc8e9f30da"
CSV_FILE_PATH = "air_quality_feature_store.csv"

# Function to fetch data from the API
def fetch_air_quality_data(timestamp):
    # Modify the API URL to include a timestamp (if applicable)
    # Here, we assume the API can take a timestamp parameter in a suitable format
    request_url = f"{API_URL}&timestamp={int(timestamp.timestamp())}"  # Adjust according to the actual API format
    try:
        response = requests.get(request_url)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
        
        data = response.json()
        if 'data' in data and data['data'] is not None:
            iaqi = data['data'].get('iaqi', {})
            return {
                "location_id": "delhi_mandir_marg",
                "event_timestamp": timestamp,  # Use the passed timestamp
                "temperature": iaqi.get('t', {}).get('v', None),
                "humidity": iaqi.get('h', {}).get('v', None),
                "pressure": iaqi.get('p', {}).get('v', None),
                "wind_speed": iaqi.get('w', {}).get('v', None),
                "wind_direction": iaqi.get('wd', {}).get('v', None),
                "pm25": iaqi.get('pm25', {}).get('v', None),
                "pm10": iaqi.get('pm10', {}).get('v', None),
                "no2": iaqi.get('no2', {}).get('v', None),
                "so2": iaqi.get('so2', {}).get('v', None),
                "o3": iaqi.get('o3', {}).get('v', None),
                "co": iaqi.get('co', {}).get('v', None)
            }
        else:
            print("No data found in API response.")
            return None
    except requests.exceptions.RequestException as e:
        print("Error fetching data:", e)
        return None

# Function to append data to CSV
def append_data_to_csv(data_dict, file_path):
    # Check if the file exists to write the header only if needed
    file_exists = os.path.isfile(file_path)
    df = pd.DataFrame([data_dict])
    
    try:
        if not file_exists:
            df.to_csv(file_path, index=False, mode='w')  # Create new file with header
        else:
            df.to_csv(file_path, index=False, mode='a', header=False)  # Append without header
    except Exception as e:
        print("Error writing to CSV:", e)

# Define the date range for historical data (e.g., past 7 days)
start_date = datetime.utcnow() - timedelta(days=7200)  # Adjust the range as needed
end_date = datetime.utcnow()

# Generate timestamps for each hour in the date range
current_date = start_date
while current_date <= end_date:
    new_data = fetch_air_quality_data(current_date)
    if new_data:
        append_data_to_csv(new_data, CSV_FILE_PATH)
        print("New data appended for date:", new_data["event_timestamp"])
    
    # Move to the next hour
    current_date += timedelta(hours=1)


New data appended for date: 2005-02-17 06:55:04.631370
New data appended for date: 2005-02-17 07:55:04.631370
New data appended for date: 2005-02-17 08:55:04.631370
New data appended for date: 2005-02-17 09:55:04.631370
New data appended for date: 2005-02-17 10:55:04.631370
New data appended for date: 2005-02-17 11:55:04.631370
New data appended for date: 2005-02-17 12:55:04.631370
New data appended for date: 2005-02-17 13:55:04.631370
New data appended for date: 2005-02-17 14:55:04.631370
New data appended for date: 2005-02-17 15:55:04.631370
New data appended for date: 2005-02-17 16:55:04.631370
New data appended for date: 2005-02-17 17:55:04.631370
New data appended for date: 2005-02-17 18:55:04.631370
New data appended for date: 2005-02-17 19:55:04.631370
New data appended for date: 2005-02-17 20:55:04.631370
New data appended for date: 2005-02-17 21:55:04.631370
New data appended for date: 2005-02-17 22:55:04.631370
New data appended for date: 2005-02-17 23:55:04.631370


KeyboardInterrupt: 

In [13]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import os

# Define the API endpoint and token
API_URL = "https://api.waqi.info/feed/here/?token=f72812a3dce2b2328992b2b06e42f4bc8e9f30da"
CSV_FILE_PATH = "air_quality_feature_store.csv"

# Function to fetch data from the API
def fetch_air_quality_data(timestamp):
    
    request_url = f"{API_URL}&timestamp={int(timestamp.timestamp())}"  # Adjust according to the actual API format
    try:
        response = requests.get(request_url)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
        
        data = response.json()
        if 'data' in data and data['data'] is not None:
            iaqi = data['data'].get('iaqi', {})
            # Extract features
            features = {
                "temperature": iaqi.get("t", {}).get("v", None),
                "humidity": iaqi.get("h", {}).get("v", None),
                "pressure": iaqi.get("p", {}).get("v", None),
                "wind_speed": iaqi.get("w", {}).get("v", None),
                "wind_direction": iaqi.get("wd", {}).get("v", None),
                "pm25": iaqi.get("pm25", {}).get("v", None),
                "pm10": iaqi.get("pm10", {}).get("v", None),
                "no2": iaqi.get("no2", {}).get("v", None),
                "so2": iaqi.get("so2", {}).get("v", None),
                "o3": iaqi.get("o3", {}).get("v", None),
                "co": iaqi.get("co", {}).get("v", None)
            }
            # Extract target (AQI)
            target = data['data'].get("aqi", None)

            return {
                "location_id": "delhi_mandir_marg",
                "event_timestamp": timestamp,  # Use the passed timestamp
                **features,  # Unpack features into the return dictionary
                "target": target  # Include target in the return dictionary
            }
        else:
            print("No data found in API response.")
            return None
    except requests.exceptions.RequestException as e:
        print("Error fetching data:", e)
        return None

# Function to append data to CSV
def append_data_to_csv(data_dict, file_path):
    # Check if the file exists to write the header only if needed
    file_exists = os.path.isfile(file_path)
    df = pd.DataFrame([data_dict])
    
    try:
        if not file_exists:
            df.to_csv(file_path, index=False, mode='w')  # Create new file with header
        else:
            df.to_csv(file_path, index=False, mode='a', header=False)  # Append without header
    except Exception as e:
        print("Error writing to CSV:", e)

# Define the date range for historical data (e.g., past 7 days)
start_date = datetime.utcnow() - timedelta(days=7200)  # Adjust the range as needed
end_date = datetime.utcnow()

# Generate timestamps for each hour in the date range
current_date = start_date
while current_date <= end_date:
    new_data = fetch_air_quality_data(current_date)
    if new_data:
        append_data_to_csv(new_data, CSV_FILE_PATH)
        print("New data appended for date:", new_data["event_timestamp"])
    
    # Move to the next hour
    current_date += timedelta(hours=1)


New data appended for date: 2005-02-17 06:57:54.304822
New data appended for date: 2005-02-17 07:57:54.304822
New data appended for date: 2005-02-17 08:57:54.304822
New data appended for date: 2005-02-17 09:57:54.304822
New data appended for date: 2005-02-17 10:57:54.304822
New data appended for date: 2005-02-17 11:57:54.304822
New data appended for date: 2005-02-17 12:57:54.304822
New data appended for date: 2005-02-17 13:57:54.304822
New data appended for date: 2005-02-17 14:57:54.304822
New data appended for date: 2005-02-17 15:57:54.304822
New data appended for date: 2005-02-17 16:57:54.304822
New data appended for date: 2005-02-17 17:57:54.304822
New data appended for date: 2005-02-17 18:57:54.304822
New data appended for date: 2005-02-17 19:57:54.304822
New data appended for date: 2005-02-17 20:57:54.304822
New data appended for date: 2005-02-17 21:57:54.304822
New data appended for date: 2005-02-17 22:57:54.304822
New data appended for date: 2005-02-17 23:57:54.304822
New data a

KeyboardInterrupt: 

In [26]:
import requests

# Replace with your actual API token
API_TOKEN = "f72812a3dce2b2328992b2b06e42f4bc8e9f30da"

# List of known cities/neighborhoods in Delhi
cities = ["New Delhi", "Delhi"]

# Iterate over the list of cities and fetch air quality data
for city in cities:
    request_url = f"https://api.waqi.info/search/?token={API_TOKEN}&keyword={city}"
    
    response = requests.get(request_url)
    response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)
    
    data = response.json()
    
    # Check if any data was returned
    if data["data"]:
        print(f"Air Quality data for {city}:")
        for entry in data["data"]:
            print(f"  Station Name: {entry['station']['name']}, "
                  f"Location: {entry['station']['geo']}, "
                  f"AQI: {entry['aqi']}")
    else:
        print(f"No data found for {city}.")


Air Quality data for New Delhi:
  Station Name: Delhi Institute of Tool Engineering, Wazirpur, Delhi, Delhi, India, Location: [28.700505, 77.165603], AQI: 499
  Station Name: Satyawati College, Delhi, Delhi, India, Location: [28.69572, 77.181295], AQI: 493
  Station Name: ITI Shahdra, Jhilmil Industrial Area, Delhi, Delhi, India, Location: [28.672114, 77.313832], AQI: 429
  Station Name: Sonia Vihar Water Treatment Plant DJB, Delhi, Delhi, India, Location: [28.710066, 77.24622], AQI: 414
  Station Name: PGDAV College, Sriniwaspuri, Delhi, Delhi, India, Location: [28.566827, 77.251418], AQI: 393
  Station Name: Pusa, Delhi, Delhi, India, Location: [28.636997, 77.172248], AQI: 384
  Station Name: R.K. Puram, Delhi, Delhi, India, Location: [28.5648, 77.1744], AQI: 353
  Station Name: Major Dhyan Chand National Stadium, Delhi, Delhi, India, Location: [28.612498, 77.237388], AQI: 335
  Station Name: Mandir Marg, Delhi, Delhi, India, Location: [28.6341, 77.2005], AQI: 303
  Station Name: New

In [24]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import os

# Define the API endpoint and token
API_URL = "https://api.waqi.info/feed/"
API_TOKEN = "f72812a3dce2b2328992b2b06e42f4bc8e9f30da"
CSV_FILE_PATH = "air_quality_feature_store.csv"
CITIES = [
    "gurugram/sector-51",
    "delhi/pooth_khurd",
    "delhi/bramprakash_ayurvedic_hospital",
    "delhi/delhi_institute_of_tool_engineering",
    "gurugram/teri_gram",
    "delhi/dr_karni_singh_shooting_range",
    "delhi/major_dhyan_chand_national_stadium",
    "delhi/national_institute_of_malaria_research",
    "delhi/jawaharlal_nehru_stadium",
    "delhi/narela",
    "delhi/iti_shahdra",
    "ghaziabad/loni",
    "delhi/mundka",
    "delhi/mother_dairy_plant",
    "delhi/pgdav_college",
    "delhi/iti_jahangirpuri",
    "delhi/satyawati_college",
    "faridabad/sector_30",
    "gurugram/nise_gwal_pahari",
    "delhi/rk_puram",
    "delhi/pusa",
    "delhi/sri_auribindo_marg",
    "delhi/anand_vihar",
    "delhi/alipur",
    "bahadurgarh/arya_nagar",
    "gurugram/vikas_sadan",
    "delhi/mandir_marg",
    "delhi/new_delhi_us_embassy",
    "delhi/sonia_vihar_water_treatment_plant",
    "delhi/punjabi_bagh",
    "delhi/shaheed_sukhdev_college_of_business_studies",
    "faridabad/sector16A",
    "delhi/dite_okhla",
    "noida/sector_125",
    "noida/sector-1"
]


# Function to fetch data from the API for a given city
def fetch_air_quality_data(city, timestamp):
    request_url = f"{API_URL}{city}/?token={API_TOKEN}&timestamp={int(timestamp.timestamp())}"
    try:
        response = requests.get(request_url)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)

        # Log the raw response text for debugging
        print(f"Response for {city} at {timestamp}: {response.text}")

        data = response.json()
        if isinstance(data, dict) and 'data' in data and data['data'] is not None:
            iaqi = data['data'].get('iaqi', {})
            # Extract features
            features = {
                "temperature": iaqi.get("t", {}).get("v", None),
                "humidity": iaqi.get("h", {}).get("v", None),
                "pressure": iaqi.get("p", {}).get("v", None),
                "wind_speed": iaqi.get("w", {}).get("v", None),
                "wind_direction": iaqi.get("wd", {}).get("v", None),
                "pm25": iaqi.get("pm25", {}).get("v", None),
                "pm10": iaqi.get("pm10", {}).get("v", None),
                "no2": iaqi.get("no2", {}).get("v", None),
                "so2": iaqi.get("so2", {}).get("v", None),
                "o3": iaqi.get("o3", {}).get("v", None),
                "co": iaqi.get("co", {}).get("v", None)
            }
            # Extract target (AQI) and rename it to 'aqi'
            aqi = data['data'].get("aqi", None)

            return {
                "location_id": city,  # Use the city name as the location ID
                "city": city.split("/")[1].replace("_", " "),  # Extract and format the city name
                "event_timestamp": timestamp,  # Use the passed timestamp
                **features,  # Unpack features into the return dictionary
                "aqi": aqi  # Include aqi in the return dictionary
            }
        else:
            print(f"Unexpected data format for {city} at {timestamp}: {data}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {city}: {e}")
        return None
    except ValueError as ve:
        print(f"Error decoding JSON for {city} at {timestamp}: {ve}")
        return None

# Function to append data to CSV
def append_data_to_csv(data_dict, file_path):
    file_exists = os.path.isfile(file_path)
    df = pd.DataFrame([data_dict])
    
    try:
        if not file_exists:
            df.to_csv(file_path, index=False, mode='w')  # Create new file with header
        else:
            df.to_csv(file_path, index=False, mode='a', header=False)  # Append without header
    except Exception as e:
        print("Error writing to CSV:", e)

# Define the date range for historical data (e.g., past 740 days)
start_date = datetime.utcnow() - timedelta(days=740)
end_date = datetime.utcnow()

# Generate timestamps for each hour in the date range
current_date = start_date
while current_date <= end_date:
    for city in CITIES:
        new_data = fetch_air_quality_data(city, current_date)
        if new_data:
            append_data_to_csv(new_data, CSV_FILE_PATH)
            print(f"New data appended for {new_data['city']} at {new_data['event_timestamp']}")
    
    # Move to the next hour
    current_date += timedelta(hours=1)


Response for delhi/pooth_khurd at 2022-10-26 07:14:08.689059: {"status":"error","data":"Unknown station"}


AttributeError: 'str' object has no attribute 'get'

In [25]:
# Function to fetch data from the API for a given city
def fetch_air_quality_data(city, timestamp):
    request_url = f"{API_URL}{city}/?token={API_TOKEN}&timestamp={int(timestamp.timestamp())}"
    try:
        response = requests.get(request_url)
        response.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)

        # Log the raw response text for debugging
        print(f"Response for {city} at {timestamp}: {response.text}")

        data = response.json()
        
        # Check if the status is 'error' and handle it
        if data.get("status") == "error":
            print(f"API Error for {city} at {timestamp}: {data['data']}")
            return None

        if isinstance(data, dict) and 'data' in data and data['data'] is not None:
            iaqi = data['data'].get('iaqi', {})
            # Extract features
            features = {
                "temperature": iaqi.get("t", {}).get("v", None),
                "humidity": iaqi.get("h", {}).get("v", None),
                "pressure": iaqi.get("p", {}).get("v", None),
                "wind_speed": iaqi.get("w", {}).get("v", None),
                "wind_direction": iaqi.get("wd", {}).get("v", None),
                "pm25": iaqi.get("pm25", {}).get("v", None),
                "pm10": iaqi.get("pm10", {}).get("v", None),
                "no2": iaqi.get("no2", {}).get("v", None),
                "so2": iaqi.get("so2", {}).get("v", None),
                "o3": iaqi.get("o3", {}).get("v", None),
                "co": iaqi.get("co", {}).get("v", None)
            }
            # Extract target (AQI) and rename it to 'aqi'
            aqi = data['data'].get("aqi", None)

            return {
                "location_id": city,  # Use the city name as the location ID
                "city": city.split("/")[1].replace("_", " "),  # Extract and format the city name
                "event_timestamp": timestamp,  # Use the passed timestamp
                **features,  # Unpack features into the return dictionary
                "aqi": aqi  # Include aqi in the return dictionary
            }
        else:
            print(f"Unexpected data format for {city} at {timestamp}: {data}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {city}: {e}")
        return None
    except ValueError as ve:
        print(f"Error decoding JSON for {city} at {timestamp}: {ve}")
        return None
